**[MDC-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160703)
tf.set_random_seed(20160703)

**[MDC-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[MDC-03]** フィルターに対応する Variable を用意して、入力データにフィルターとプーリング層を適用する計算式を定義します。

In [4]:
num_filters = 16

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv = tf.Variable(tf.truncated_normal([5,5,1,num_filters],
                                         stddev=0.1))
h_conv = tf.nn.conv2d(x_image, W_conv,
                      strides=[1,1,1,1], padding='SAME')
h_pool =tf.nn.max_pool(h_conv, ksize=[1,2,2,1],
                       strides=[1,2,2,1], padding='SAME')

**[MDC-04]** プーリング層からの出力を全結合層を経由してソフトマックス関数に入力する計算式を定義します。

In [5]:
h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters])

num_units1 = 14*14*num_filters
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.zeros([num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool_flat, w2) + b2)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0)

**[MDC-05]** 誤差関数 loss、トレーニングアルゴリズム train_step、正解率 accuracy を定義します。

In [6]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[MDC-06]** セッションを用意して、Variable を初期化します。

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

**[MDC-07]** パラメーターの最適化を4000回繰り返します。

最終的に、テストセットに対して約98%の正解率が得られます。

In [8]:
i = 0
for _ in range(4000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, t: batch_ts})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:mnist.test.images, t:mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        saver.save(sess, 'mdc_session', global_step=i)

Step: 100, Loss: 2504.719727, Accuracy: 0.925000
Step: 200, Loss: 1767.047607, Accuracy: 0.947400
Step: 300, Loss: 1471.945312, Accuracy: 0.957600
Step: 400, Loss: 1416.925049, Accuracy: 0.958400
Step: 500, Loss: 1154.307739, Accuracy: 0.962300
Step: 600, Loss: 1061.395386, Accuracy: 0.967000
Step: 700, Loss: 883.334229, Accuracy: 0.974900
Step: 800, Loss: 960.722534, Accuracy: 0.970900
Step: 900, Loss: 903.904236, Accuracy: 0.972200
Step: 1000, Loss: 771.594727, Accuracy: 0.977100
Step: 1100, Loss: 882.415527, Accuracy: 0.974300
Step: 1200, Loss: 717.312988, Accuracy: 0.976400
Step: 1300, Loss: 720.271423, Accuracy: 0.977700
Step: 1400, Loss: 683.606934, Accuracy: 0.978300
Step: 1500, Loss: 778.011902, Accuracy: 0.975700
Step: 1600, Loss: 704.136780, Accuracy: 0.979200
Step: 1700, Loss: 660.429688, Accuracy: 0.980100
Step: 1800, Loss: 704.402100, Accuracy: 0.978900
Step: 1900, Loss: 740.106323, Accuracy: 0.976200
Step: 2000, Loss: 665.510193, Accuracy: 0.980600
Step: 2100, Loss: 646.1

**[MDC-08]** セッション情報を保存したファイルが生成されていることを確認します。

In [9]:
!ls mdc_session*

mdc_session-100.data-00000-of-00001   mdc_session-2800.data-00000-of-00001
mdc_session-100.index		      mdc_session-2800.index
mdc_session-100.meta		      mdc_session-2800.meta
mdc_session-1000.data-00000-of-00001  mdc_session-2900.data-00000-of-00001
mdc_session-1000.index		      mdc_session-2900.index
mdc_session-1000.meta		      mdc_session-2900.meta
mdc_session-1100.data-00000-of-00001  mdc_session-300.data-00000-of-00001
mdc_session-1100.index		      mdc_session-300.index
mdc_session-1100.meta		      mdc_session-300.meta
mdc_session-1200.data-00000-of-00001  mdc_session-3000.data-00000-of-00001
mdc_session-1200.index		      mdc_session-3000.index
mdc_session-1200.meta		      mdc_session-3000.meta
mdc_session-1300.data-00000-of-00001  mdc_session-3100.data-00000-of-00001
mdc_session-1300.index		      mdc_session-3100.index
mdc_session-1300.meta		      mdc_session-3100.meta
mdc_session-1400.data-00000-of-00001  mdc_session-3200.data-00000-of-00001
mdc_session-1400.index		      mdc_s